In [1]:
# basic imports
import pandas as pd
import sys
import numpy as np
import pandas as pd
import scanpy as sc

# add `Tangram` to path
import tangram as tg
import os

In [2]:
## setting for STARmap
sc_file_path = "/home/comp/cszrwang/data/Melanoma/sc_mela.h5ad"
spatial_file_path = "/home/comp/cszrwang/data/Melanoma/st_mela.h5ad"
celltype_key = 'celltype'
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/Melanoma//Tangram/"
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)

In [3]:
ad_sc = sc.read_h5ad(sc_file_path)
ad_sp = sc.read_h5ad(spatial_file_path)

# use raw count both of scrna and spatial
sc.pp.normalize_total(ad_sc)
celltype_counts = ad_sc.obs[celltype_key].value_counts()
celltype_drop = celltype_counts.index[celltype_counts < 2]
print(f'Drop celltype {list(celltype_drop)} contain less 2 sample')
ad_sc = ad_sc[~ad_sc.obs[celltype_key].isin(celltype_drop),].copy()
sc.tl.rank_genes_groups(ad_sc, groupby=celltype_key, use_raw=False)
markers_df = pd.DataFrame(ad_sc.uns["rank_genes_groups"]["names"]).iloc[0:200, :]
print(markers_df)
genes_sc = np.unique(markers_df.melt().value.values)
print(genes_sc)
genes_st = ad_sp.var_names.values
genes = list(set(genes_sc).intersection(set(genes_st)))

Drop celltype [] contain less 2 sample
            B     CAF    Endo    Macro       Mal        NK          T
0       CD79A  COL1A1  IGFBP7   TYROBP     S100B      PRF1       CD3D
1     HLA-DRA   CALD1   HYAL2   FCER1G  SERPINE2      NKG7        CD2
2       MS4A1  COL1A2     APP    IFI30      PLP1      CTSW       IL32
3    HLA-DPA1   FSTL1  IFITM3     AIF1     PRAME     IL2RB       CD3E
4        CD74   SPARC   EGFL7     CD68     GPNMB     KLRD1       CD3G
..        ...     ...     ...      ...       ...       ...        ...
195     DAPP1     VCL     A2M     CSTA     CRTAP    L2HGDH      AKAP5
196     FCRL5  AMOTL2   FLOT1  SH3BGRL    ARPC1A     KDM4C  RAMP2-AS1
197     PTPN6  PGRMC1  GPR146   SAMHD1     MGST3  TNFRSF18        ID2
198      GRB2    UGDH  GPRC5B     LAT2     CSAG1       SHE   ARHGAP15
199      RPS7   FSCN1    DLC1     DOK3     AIF1L     ITGAM      IKZF1

[200 rows x 7 columns]
['A2M' 'AAK1' 'ABCA1' ... 'ZNF521' 'ZNF793' 'ZYX']


In [4]:
tg.pp_adatas(ad_sc, ad_sp, genes=genes)

ad_map = tg.map_cells_to_space(
                   ad_sc,
                   ad_sp,
                   mode='clusters',
                   cluster_label=celltype_key)

# ad_map = tg.map_cells_to_space(
#                    ad_sc,
#                    ad_sp,
#                    mode="constrained",
#                    target_count=ad_sp.obs.cell_count.sum(),
#                    density_prior=np.array(ad_sp.obs.cell_count) / ad_sp.obs.cell_count.sum(),
#                    num_epochs=1000,
#                    cluster_label=celltype_key)

tg.project_cell_annotations(ad_map, ad_sp, annotation=celltype_key)

celltype_density = ad_sp.obsm['tangram_ct_pred']
celltype_density = (celltype_density.T/celltype_density.sum(axis=1)).T

celltype_density.to_csv(output_file_path + '/Tangram_result.csv')

INFO:root:1145 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:14517 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1145 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.323, KL reg: 0.460
Score: 0.566, KL reg: 0.003
Score: 0.567, KL reg: 0.003
Score: 0.568, KL reg: 0.003
Score: 0.568, KL reg: 0.003
Score: 0.568, KL reg: 0.003
Score: 0.568, KL reg: 0.003
Score: 0.568, KL reg: 0.003
Score: 0.568, KL reg: 0.003
Score: 0.568, KL reg: 0.003


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
